In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("william2020/marine-microplastics")

print("Path to dataset files:", path)

c:\Users\isaw9\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\isaw9\.cache\kagglehub\datasets\william2020\marine-microplastics\versions\1


In [2]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [3]:
df_mp = pd.read_csv("../Archivos/Marine_Microplastics.csv")
df_mp.head()

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,10/17/2006 12:00:00 AM,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,10/17/2018 12:00:00 AM,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,1/3/2015 12:00:00 AM,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000


In [4]:
df_mp.isna().sum()/df_mp.shape[0]*100

OBJECTID             0.000000
Oceans               1.326805
Regions             56.499388
SubRegions          93.600979
Sampling Method      0.000000
Measurement         28.455324
Unit                 0.000000
Density Range        0.000000
Density Class        0.000000
Short Reference      0.000000
Long Reference       0.000000
DOI                  0.000000
Organization         0.000000
Keywords             0.088127
Accession Number     0.000000
Accession Link       0.000000
Latitude             0.000000
Longitude            0.000000
Date                 0.000000
GlobalID             0.000000
x                    0.000000
y                    0.000000
dtype: float64

In [5]:
df_mp.shape

(20425, 22)

In [6]:
df_mp['Measurement'].describe()

count     14613.000000
mean        161.983976
std        2198.862484
min           0.000000
25%           0.000000
50%           0.008640
75%           0.128412
max      110480.000000
Name: Measurement, dtype: float64

In [7]:
# En primer lugar se realizará una visión de nulos y duplicados.

df_mp.duplicated().sum() # No hay duplicados 😀

np.int64(0)

In [8]:
# Gestión de nulos para:Oceans, Regions, Subregions y Mesurement.

In [9]:
df_mp['Regions'].nunique()

47

In [10]:
# Limpiar espacios en blanco adicionales y normalizar el texto
df_mp['Regions'] = df_mp['Regions'].str.strip()  # Eliminar espacios antes y después
df_mp['Regions'] = df_mp['Regions'].str.lower()  # Convertir a minúsculas (opcional, solo si es necesario)

# Verificar los valores únicos después de la limpieza
print(df_mp['Regions'].unique())

#Porque Rio de la plata aparecía repetido

[nan 'caribbean sea' 'mediterranean sea' 'north sea'
 'inner seas off the west coast of scotland' 'new york bight'
 'gulf of mexico' 'gulf of california' 'celtic sea'
 'coastal waters of florida'
 'coastal waters of southeast alaska and british columbia'
 'gulf of alaska' 'stellwagen bank national marine sanctuary'
 "irish sea and st. george's channel" 'norwegian sea' 'kattegat'
 'english channel' 'gulf of st. lawrence' 'baltic sea'
 'papahanaumokuakea marine national monument' 'bay of biscay'
 'northwestern passages' 'monterey bay national marine sanctuary'
 'skagerrak strait'
 'the coastal waters of southeast alaska and british columbia'
 'beaufort sea' 'davis strait' 'greenland sea' 'bay of fundy'
 'gulf of bothnia' 'baffin bay' 'black sea'
 'florida keys national marine sanctuary' 'labrador sea' 'hudson strait'
 'mozambique channel' 'red sea' 'olympic coast national marine sanctuary'
 'rio de la plata' 'channel islands national marine sanctuary'
 'chukchi sea' 'bering sea' 'barents

In [11]:
df_mp['Regions'].nunique()

46

In [12]:
# Registros completos donde ambos son nulos

df_mp[df_mp['Regions'].isnull() & df_mp['SubRegions'].isnull()]


,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,10/17/2006 12:00:00 AM,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,10/17/2018 12:00:00 AM,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,1/3/2015 12:00:00 AM,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20418,7852,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,33.060000,-148.130000,10/16/2009 12:00:00 AM,12514012-1683-44f0-ab2b-2ec269e2e906,-148.130000,33.060000
20419,7557,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,31.540000,-143.820000,7/6/2008 12:00:00 AM,86bb49ef-27c1-4cba-930a-796315290491,-143.820000,31.540000
20420,9366,Atlantic Ocean,NaN,NaN,Grab sample,0.011,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,17.100000,-47.550000,2/1/2015 12:00:00 AM,fd4db6d6-aed7-48b1-a22b-402557112e2f,-47.550000,17.100000
20422,6043,Atlantic Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2010,"Law, K.L., S. Morét-Ferguson, N.A. Maximenko, ...",https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,24.140000,-81.980000,5/1/2008 12:00:00 AM,0fd72d83-cd27-42e1-b63a-b015c23b28d2,-81.980000,24.140000


In [13]:
# Lo primero quizás los océanos y Mesurement. Los océanos será facil con la latitud y la longitud

In [14]:
#Relacionar oceanos con Longitud y Latitud
df_mp[df_mp['Oceans'].notnull()].groupby('Oceans')[['Latitude', 'Longitude']].mean().sort_values('Latitude')

,Latitude,Longitude
Oceans,,
Indian Ocean,-26.203959,28.731356
Pacific Ocean,23.412917,-137.379422
Atlantic Ocean,28.981572,-65.054741
Arctic Ocean,77.023468,-40.293646


In [15]:
# Primero, voy a crear una columna nueva, para crear un map en los oceanos, y poder imputar con latitud

df_mp["Oceans"].unique()

array(['Atlantic Ocean', 'Pacific Ocean', 'Arctic Ocean', nan,
       'Indian Ocean'], dtype=object)

In [16]:
dicc ={'Atlantic Ocean':1,'Pacific Ocean':2,'Arctic Ocean':3,'Indian Ocean':4, "nan":None}

In [17]:
df_mp['Oceans_cat']=df_mp['Oceans'].map(dicc)

In [18]:
df_mp

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y,Oceans_cat
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.0180,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000,1.0
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.0000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000,2.0
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.0130,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,10/17/2006 12:00:00 AM,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000,2.0
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.0000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,10/17/2018 12:00:00 AM,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825,1.0
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.0010,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,1/3/2015 12:00:00 AM,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20420,9366,Atlantic Ocean,NaN,NaN,Grab sample,0.0110,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,17.100000,-47.550000,2/1/2015 12:00:00 AM,fd4db6d6-aed7-48b1-a22b-402557112e2f,-47.550000,17.100000,1.0
20421,4683,Atlantic Ocean,caribbean sea,NaN,Neuston net,0.0216,pieces/m3,0.005-1,Medium,Law et al.2010,"Law, K.L., S. Morét-Ferguson, N.A. Maximenko, ...",https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,21.390000,-81.960000,3/14/2001 12:00:00 AM,83a8208e-2d06-4ad9-bcd9-81e29d57e0aa,-81.960000,21.390000,1.0
20422,6043,Atlantic Ocean,NaN,NaN,Neuston net,0.0000,pieces/m3,0-0.0005,Very Low,Law et al.2010,"Law, K.L., S. Morét-Ferguson, N.A. Maximenko, ...",https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,24.140000,-81.980000,5/1/2008 12:00:00 AM,0fd72d83-cd27-42e1-b63a-b015c23b28d2,-81.980000,24.140000,1.0
20423,20169,Atlantic Ocean,NaN,NaN,Hand picking,NaN,pieces/10 mins,40-200,High,Tunnell et al. 2020,"Tunnell, Jace W.; Dunning, Kelly H.; Scheef, L...",https://doi.org/10.1016/j.marpolbul.2019.110794,University of Texas Marine Science Institute,Nurdle Patrol,259486,https://www.ncei.noaa.gov/access/metadata/land...,39.924800,-75.127200,10/28/2021 12:00:00 AM,fc84069e-c782-4c3d-bf6b-265535969d8e,-75.127200,39.924800,1.0


In [19]:
df_mp[df_mp['Oceans_cat'].notnull()].groupby('Oceans_cat')[['Latitude', 'Longitude']].mean().sort_values('Latitude')

,Latitude,Longitude
Oceans_cat,,
4.0,-26.203959,28.731356
2.0,23.412917,-137.379422
1.0,28.981572,-65.054741
3.0,77.023468,-40.293646


In [20]:
imputer_knn = KNNImputer(n_neighbors=3)
imputado = imputer_knn.fit_transform(df_mp[["Oceans_cat", "Latitude"]])

In [21]:
pd.DataFrame(imputado)

,0,1
0,1.0,-31.696000
1,2.0,6.350000
2,2.0,0.500000
3,1.0,0.631825
4,2.0,16.623000
...,...,...
20420,1.0,17.100000
20421,1.0,21.390000
20422,1.0,24.140000
20423,1.0,39.924800


In [22]:
df_mp[["Ocean_cat_knn", "Latitude_knn"]] = imputado

In [23]:
df_mp.head()

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y,Oceans_cat,Ocean_cat_knn,Latitude_knn
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000,1.0,1.0,-31.696000
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000,2.0,2.0,6.350000
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,10/17/2006 12:00:00 AM,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000,2.0,2.0,0.500000
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,10/17/2018 12:00:00 AM,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825,1.0,1.0,0.631825
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,1/3/2015 12:00:00 AM,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000,2.0,2.0,16.623000


In [24]:
df_mp = df_mp.drop(columns=["Oceans_cat", "Latitude_knn"])

In [25]:
df_mp.head()

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y,Ocean_cat_knn
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000,1.0
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000,2.0
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,10/17/2006 12:00:00 AM,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000,2.0
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,10/17/2018 12:00:00 AM,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825,1.0
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,1/3/2015 12:00:00 AM,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000,2.0


In [26]:
dicc2 ={1:'Atlantic Ocean',2:"Pacific Ocean",3:'Arctic Ocean',4:'Indian Ocean'}

In [27]:
df_mp['Oceans'] = df_mp['Oceans'].fillna(df_mp['Ocean_cat_knn'].round().astype(int).map(dicc2))

In [28]:
df_mp.head()

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y,Ocean_cat_knn
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000,1.0
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000,2.0
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,10/17/2006 12:00:00 AM,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000,2.0
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,10/17/2018 12:00:00 AM,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825,1.0
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,1/3/2015 12:00:00 AM,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000,2.0


In [29]:
df_mp=df_mp.drop(columns=["Ocean_cat_knn"])

In [30]:
df_mp.isna().sum()/df_mp.shape[0]*100

OBJECTID             0.000000
Oceans               0.000000
Regions             56.499388
SubRegions          93.600979
Sampling Method      0.000000
Measurement         28.455324
Unit                 0.000000
Density Range        0.000000
Density Class        0.000000
Short Reference      0.000000
Long Reference       0.000000
DOI                  0.000000
Organization         0.000000
Keywords             0.088127
Accession Number     0.000000
Accession Link       0.000000
Latitude             0.000000
Longitude            0.000000
Date                 0.000000
GlobalID             0.000000
x                    0.000000
y                    0.000000
dtype: float64

In [31]:
df_mp["Sampling Method"].unique()

array(['Grab sample', 'Neuston net', 'Manta net', 'Aluminum bucket',
       'Megacorer', 'Intake seawater pump', 'Hand picking',
       'PVC cylinder', 'Van Dorn sampler', 'Remotely operated vehicle',
       'Metal spoon', 'AVANI net', 'Plankton net', 'Day grab',
       'Stainless steel spoon', 'Shipek grab sampler',
       'Petite Ponar benthic grab', 'Van Veen grab sampler',
       'CTD rosette sampler'], dtype=object)

In [32]:
# Gestión de nulos de Measurement
df_mp.head()

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,10/17/2006 12:00:00 AM,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,10/17/2018 12:00:00 AM,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,1/3/2015 12:00:00 AM,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000


In [33]:
# Ver si hay relación entre Sampling Method y Unit.

In [34]:
df_mp["Unit"].unique()

array(['pieces/m3', 'pieces kg-1 d.w.', 'pieces/10 mins'], dtype=object)

In [35]:
# saber a que método corresponden las medidas
# Tabla de contingencia entre Métodos y Unidades de Medida
contingencia = pd.crosstab(df_mp['Sampling Method'], df_mp['Unit'], margins=True)
display(contingencia)

Unit,pieces kg-1 d.w.,pieces/10 mins,pieces/m3,All
Sampling Method,,,,
AVANI net,0,0,18,18
Aluminum bucket,0,0,57,57
CTD rosette sampler,0,0,36,36
Day grab,17,0,0,17
Grab sample,0,0,1181,1181
Hand picking,0,5812,18,5830
Intake seawater pump,0,0,181,181
Manta net,0,0,2342,2342
Megacorer,90,0,0,90


In [36]:
# Se puede observar, que excepto "hand picking" las demás tienen una unidad definida.
# Entonces, por la unidad, podemos sacar el método que se utilizó, obviando "hand picking"

#Como tenemos nulos en measurement, no en unit.

In [37]:
df_mp.groupby("Sampling Method")["Measurement"].apply(lambda x: x.isnull().sum())
# Podemos ver que todos los nulos son de Hand picking, "recogido a mano"--> No relevante --> No imputación

Sampling Method
AVANI net                       0
Aluminum bucket                 0
CTD rosette sampler             0
Day grab                        0
Grab sample                     0
Hand picking                 5812
Intake seawater pump            0
Manta net                       0
Megacorer                       0
Metal spoon                     0
Neuston net                     0
PVC cylinder                    0
Petite Ponar benthic grab       0
Plankton net                    0
Remotely operated vehicle       0
Shipek grab sampler             0
Stainless steel spoon           0
Van Dorn sampler                0
Van Veen grab sampler           0
Name: Measurement, dtype: int64

In [38]:
df_mp.head(2)

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696,-48.56,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.56,-31.696
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350,-121.85,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.85,6.350


In [39]:
# Las columnas x e y, son igual que latitud y longitud. Se podrían eliminar

In [40]:
df_mp["Date"].nunique()

5997

In [41]:
# Primero en DATE, hago un split por " "

In [42]:
# Separar la columna 'Date' en fecha y hora por el espacio
# En caso de que alguna fila no tenga espacio, la función llenará con NaN
df_mp[['Date_Only', 'Time_Only']] = df_mp['Date'].str.split(' ', expand=True, n=1)

# Verificar el resultado
print(df_mp[['Date_Only', 'Time_Only']].head())


    Date_Only    Time_Only
0   8/11/2015  12:00:00 AM
1  12/18/2002  12:00:00 AM
2  10/17/2006  12:00:00 AM
3  10/17/2018  12:00:00 AM
4    1/3/2015  12:00:00 AM


In [43]:
df_mp["Time_Only"].unique()
#Todas las horas son las 12 AM, podemos prescindir de ese dato:

array(['12:00:00 AM'], dtype=object)

In [44]:
df_mp['Date'] = pd.to_datetime(df_mp['Date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

In [45]:
# Extraer el año
df_mp['Year'] = df_mp['Date'].dt.year

# Extraer el mes
df_mp['Month'] = df_mp['Date'].dt.month

# Extraer la hora
df_mp['Hour'] = df_mp['Date'].dt.hour


In [46]:
df_mp.head()

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y,Date_Only,Time_Only,Year,Month,Hour
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,2015-08-11,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000,8/11/2015,12:00:00 AM,2015,8,0
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,2002-12-18,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000,12/18/2002,12:00:00 AM,2002,12,0
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,2006-10-17,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000,10/17/2006,12:00:00 AM,2006,10,0
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,2018-10-17,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825,10/17/2018,12:00:00 AM,2018,10,0
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,2015-01-03,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000,1/3/2015,12:00:00 AM,2015,1,0


In [47]:
# Time_Only y Hour, las podemos borrar

# Siempre 12:00 AM

df_mp = df_mp.drop(columns=["Time_Only", "Hour","x","y",'SubRegions'])

In [48]:
df_mp.head(2)

,OBJECTID,Oceans,Regions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,Date_Only,Year,Month
0,9676,Atlantic Ocean,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696,-48.56,2015-08-11,a77121b2-e113-444e-82d9-7af11d62fdd2,8/11/2015,2015,8
1,6427,Pacific Ocean,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350,-121.85,2002-12-18,be27c450-02ca-4261-8d89-cae21108e6cc,12/18/2002,2002,12


In [49]:
# Obtener los valores únicos de la columna 'Year' y ordenarlos
sorted_years = sorted(df_mp['Year'].unique())

# Mostrar los valores ordenados
print(sorted_years)

# Los años van de 1972 al 2002

[np.int32(1972), np.int32(1973), np.int32(1986), np.int32(1987), np.int32(1989), np.int32(1990), np.int32(1991), np.int32(1992), np.int32(1993), np.int32(1994), np.int32(1995), np.int32(1996), np.int32(1997), np.int32(1998), np.int32(1999), np.int32(2000), np.int32(2001), np.int32(2002), np.int32(2003), np.int32(2004), np.int32(2005), np.int32(2006), np.int32(2007), np.int32(2008), np.int32(2009), np.int32(2010), np.int32(2011), np.int32(2012), np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022)]


In [50]:
# Sacar las regiones con la latitud y la longitud, para imputar las regiones que faltan.

# !pip install geopy

"""Geopy es una librería de Python que proporciona herramientas para interactuar con APIs
de geolocalización, como las de Google Maps, Nominatim de OpenStreetMap, y otros servicios
de geocodificación y geolocalización inversa. Permite convertir coordenadas geográficas
en direcciones (geocodificación inversa) y viceversa (geocodificación directa).
También ofrece soporte para el cálculo de distancias y la geolocalización en diferentes coordenadas."""


'Geopy es una librería de Python que proporciona herramientas para interactuar con APIs\nde geolocalización, como las de Google Maps, Nominatim de OpenStreetMap, y otros servicios\nde geocodificación y geolocalización inversa. Permite convertir coordenadas geográficas\nen direcciones (geocodificación inversa) y viceversa (geocodificación directa).\nTambién ofrece soporte para el cálculo de distancias y la geolocalización en diferentes coordenadas.'

In [51]:
# Agrupar por 'Oceans' y obtener las regiones únicas para cada océano
oceans_regions = df_mp.groupby('Oceans')['Regions'].unique()

# Mostrar el resultado
print(oceans_regions)


Oceans
Arctic Ocean      [nan, norwegian sea, northwestern passages, be...
Atlantic Ocean    [nan, caribbean sea, mediterranean sea, north ...
Indian Ocean                     [nan, mozambique channel, red sea]
Pacific Ocean     [nan, gulf of california, coastal waters of so...
Name: Regions, dtype: object


In [52]:
# Agrupar por 'Oceans' y contar las ocurrencias de cada 'Regions'
oceans_regions_count = df_mp.groupby('Oceans')['Regions'].value_counts()

# Encontrar la región más frecuente en cada océano
most_frequent_region = oceans_regions_count.groupby('Oceans').idxmax()

# Contar la frecuencia de la región más frecuente
most_frequent_count = oceans_regions_count.groupby('Oceans').max()

# Calcular el total de registros por océano
total_by_ocean = df_mp.groupby('Oceans').size()

# Calcular el porcentaje de la región más frecuente dentro de cada océano
percentage_most_frequent = (most_frequent_count / total_by_ocean) * 100

# Crear un DataFrame con la región más frecuente y su porcentaje
result = pd.DataFrame({
    'Most Frequent Region': most_frequent_region,
    'Frequency': most_frequent_count,
    'Percentage': percentage_most_frequent
})

# Mostrar el resultado
print(result)


                               Most Frequent Region  Frequency  Percentage
Oceans                                                                    
Arctic Ocean          (Arctic Ocean, greenland sea)         35   19.662921
Atlantic Ocean     (Atlantic Ocean, gulf of mexico)       4817   31.083436
Indian Ocean     (Indian Ocean, mozambique channel)          3   15.000000
Pacific Ocean   (Pacific Ocean, gulf of california)        116    2.452431


In [53]:
# No tenemos grandes porcentajes para imputar por la moda. De momento, dejamos así los datos, usaremos geopy en un futuro si es necesario

In [54]:
df_mp.head()

,OBJECTID,Oceans,Regions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,Date_Only,Year,Month
0,9676,Atlantic Ocean,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,2015-08-11,a77121b2-e113-444e-82d9-7af11d62fdd2,8/11/2015,2015,8
1,6427,Pacific Ocean,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,2002-12-18,be27c450-02ca-4261-8d89-cae21108e6cc,12/18/2002,2002,12
2,10672,Pacific Ocean,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,2006-10-17,23effcdd-35b7-4e1e-adb4-390693a287d3,10/17/2006,2006,10
3,13921,Atlantic Ocean,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,2018-10-17,16d77822-0533-4116-97b9-0bdb592f3d6e,10/17/2018,2018,10
4,9344,Pacific Ocean,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,2015-01-03,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,1/3/2015,2015,1


In [55]:
df_mp.isna().sum()/df_mp.shape[0]*100

OBJECTID             0.000000
Oceans               0.000000
Regions             56.499388
Sampling Method      0.000000
Measurement         28.455324
Unit                 0.000000
Density Range        0.000000
Density Class        0.000000
Short Reference      0.000000
Long Reference       0.000000
DOI                  0.000000
Organization         0.000000
Keywords             0.088127
Accession Number     0.000000
Accession Link       0.000000
Latitude             0.000000
Longitude            0.000000
Date                 0.000000
GlobalID             0.000000
Date_Only            0.000000
Year                 0.000000
Month                0.000000
dtype: float64

In [ ]:
# Yo creo que de momento, con este conjunto de datos limpio, podemos trabajar, para las cuestiones que se nos da:

In [56]:
# Guardar csv filtrado de Sugarcane

df_mp.to_csv('Clean_Microplastics.csv', index=False)